<span style="color:red">Abgegeben von (Name, Vorname):</span> 
Goxhufi, Driton

In [1]:
import nltk

Immer griffbereit:
- Website: https://www.nltk.org/
- Buch: https://www.nltk.org/book/
- Module: https://www.nltk.org/py-modindex.html
- Beispiele: http://www.nltk.org/howto/

# Sätze: Chunking

Zu wissen, was in einem Text die Sätze und Worte (qua Tokenisierung) und die POS-Tags der Worte (qua POS-Tagging) sind, reicht natürlich oft nicht aus, um etwas sinnvolles über den Inhalt des Textes sagen können. Wenn wir nur das Nomen *York* sehen, wissen wir noch nicht, ob [York](https://de.wikipedia.org/wiki/York) oder [New York](https://de.wikipedia.org/wiki/New_York_City) gemeint ist. Das Chunking ist eine Methode, um größere, wortübergreifende Einheiten in einem Satz zu identifizieren. 

## Was ist Chunking?

Unter Chunking versteht man die **überschneidungsfreie Gruppierung von benachbarten Wordtoken aufgrund ihres POS-Tags**. Zum Beispiel können Artikel, Adjektive und Nomen zu einem Chunk mit dem Label `NP` (für Nominalphrase) zusammengefasst werden:

<img src="https://www.nltk.org/images/chunk-segmentation.png" alt="Drawing" style="width: 500px"/>

Die Chunks werden üblicherweise mittels einer **Klammernotation** aufgeschrieben: 

    (LABEL wordform1/TAG1 wordform2/TAG2 ...)
    
Also können wir obiges Beispiel folgendermaßen aufschreiben (und fügen dabei den Satz-Chunk hinzu):

    (S
      (NP We/PRP)
      saw/VBD
      (NP the/DT yellow/JJ dog/NN))

Chunks können nicht nur Wortoken sondern auch Chunks enthalten. Dabei ist aber zu beachten, dass Chunks hinsichtlich des Labels **nicht rekursiv** sind! Das bedeutet beipielsweise, dass ein NP-Chunk kein anderes NP-Chunk enthalten darf.

Also nicht so:

    (S
      (NP We/PRP)
      saw/VBD
      (NP the/DT yellow/JJ dog/NN 
        (PP of/IN                       # PP = Präpositionalphrase
          (NP the/DT neighbor/NN))))    # Das geht nicht!
          
Sondern so: 

    (S
      (NP We/PRP)
      saw/VBD
      (NP the/DT yellow/JJ dog/NN) 
      (PP of/IN 
        (NP the/DT neighbor/NN)))

Oder noch flacher:

    (S
      (NP We/PRP)
      saw/VBD
      (NP the/DT yellow/JJ dog/NN) 
      (PP of/IN)
      (NP the/DT neighbor/NN))
      
Man nennt das Chunking deshalb auch **Shallow Parsing**, weil die resultierenden Bäume flacher sind als beim "tiefen" Parsen (zu dem wir in der nächsten Sitzung kommen werden). 

Man kann übrigens mittels der NLTK-Klasse `Tree` solche Klammerausdrücke leicht einlesen, verändern und wieder ausgeben: 

In [2]:
from nltk.tree import Tree

chunked_sent = "(S (NP We/PRP) saw/VBD (NP the/DT yellow/JJ dog/NN (PP of/IN (NP the/DT neighbor/NN))))"

print(Tree.fromstring(chunked_sent))   # Pretty-Print der Klammernotation
Tree.fromstring(chunked_sent).draw()  # öffnet ein eigenes Fenter mit Baumdarstellung

(S
  (NP We/PRP)
  saw/VBD
  (NP the/DT yellow/JJ dog/NN (PP of/IN (NP the/DT neighbor/NN))))


## Was ist ein Chunk? Und was nicht?

Chunks sind Einheiten, die sich irgendwie "sinnvoll" von ihrer Umgebung abgrenzen lassen. Tatsächlich wird nie wirklich definitiert, was "sinnvoll" bedeutet. Man orientiert sich hier stattdessen an der Konstituenten- oder Phrasenstruktur und flacht diese nach bestimmten Regeln ab.

**Beispiel:** Eine Konstituentestruktur ist oft rekursiv und sieht z.B. so aus:

    (S
      (NP We/PRP)
      saw/VBD
      (NP the/DT yellow/JJ dog/NN 
        (PP of/IN                       
          (NP the/DT neighbor/NN))))

Die Konstituentstruktur wird anhand sogenannter **Konstituententests** (auf die wir hier (noch) nicht eingehen wollen) gebildet. Solche Tests gibt es aber für Chunks meines Wissens nicht.

Stattdessen werden Chunks aus solchen Konstituenstrukturen durch Abflachung gewonnen, indem z.B. die NP und die Präpositionalphrase aus der komplexen NP herausgenommen und "angehoben" werden:

    (S
      (NP We/PRP)
      saw/VBD
      (NP the/DT yellow/JJ dog/NN) 
      (PP of/IN)
      (NP the/DT neighbor/NN))



## Methoden für das automatische Chunking

Ein Satz mit $n$ Worten enthält maximal $n+1$ Chunks, aber es gibt $2^{(n-1)}$ mögliche Chunking-Analysen. D.h. ein durchschnittlicher Satz ($n =20$) kann auf 524288 Arten gechunkt werden.  Es stellt sich also die Frage, wie man aus dieser Masse an möglichen Chunking-Analysen die richtige auswählt. 

### Vorbereitung der Trainings- und Testdaten

Für die Evaluierung der Methoden benötigen wir, ähnlich wie beim Tagging, disjunkte Trainings- und Testdaten mit Goldchunks. NLTK stellt hierfür mit `conll2000` das Chunk-Corpus des [CoNLL-2000-Shared-Task](https://www.aclweb.org/anthology/W00-0726/) bereit. Es enthält 10.948 Sätze und 259.104 Worttoken, was sicherlich ausreichend ist. <span style="color:red">(Frage am Rande: Warum ist das wohl so?)</span> <span style="color:green">Antwort: weniger POS-Tags, also Chung-Tags (weniger klassen)</span>

Für die Abfrage von `conll2000` stehen die üblichen Methoden zu Verfügung (`words()`, `sents()`, `tagged_words()`, ...). Außerdem kann man hier aber mit `chunked_sents(chunk_types=None)` die enthaltenen Chunks ausgeben, wobei mit der Option `chunk_types` die Chunks anhand des Labels eingeschränkt werden können: 

In [8]:
from nltk.corpus import conll2000
 
print(conll2000.chunked_sents(chunk_types="VP"))

[Tree('S', [('Confidence', 'NN'), ('in', 'IN'), ('the', 'DT'), ('pound', 'NN'), Tree('VP', [('is', 'VBZ'), ('widely', 'RB'), ('expected', 'VBN'), ('to', 'TO'), ('take', 'VB')]), ('another', 'DT'), ('sharp', 'JJ'), ('dive', 'NN'), ('if', 'IN'), ('trade', 'NN'), ('figures', 'NNS'), ('for', 'IN'), ('September', 'NNP'), (',', ','), ('due', 'JJ'), ('for', 'IN'), ('release', 'NN'), ('tomorrow', 'NN'), (',', ','), Tree('VP', [('fail', 'VB'), ('to', 'TO'), ('show', 'VB')]), ('a', 'DT'), ('substantial', 'JJ'), ('improvement', 'NN'), ('from', 'IN'), ('July', 'NNP'), ('and', 'CC'), ('August', 'NNP'), ("'s", 'POS'), ('near-record', 'JJ'), ('deficits', 'NNS'), ('.', '.')]), Tree('S', [('Chancellor', 'NNP'), ('of', 'IN'), ('the', 'DT'), ('Exchequer', 'NNP'), ('Nigel', 'NNP'), ('Lawson', 'NNP'), ("'s", 'POS'), ('restated', 'VBN'), ('commitment', 'NN'), ('to', 'TO'), ('a', 'DT'), ('firm', 'NN'), ('monetary', 'JJ'), ('policy', 'NN'), Tree('VP', [('has', 'VBZ'), ('helped', 'VBN'), ('to', 'TO'), ('preven

Diese Ausgabe besteht aus verschachtelten `Tree`-Objekten, die jeweils einen Mutterknoten und die Töchterknoten umfassen.

D.h. wenn wir einen Satz auswählen, wird mit `print()` die Baumstruktur des `Tree`-Objekts besser lesbar ausgegeben:

In [10]:
print(conll2000.chunked_sents(chunk_types="NP")[0])

(S
  (NP Confidence/NN)
  in/IN
  (NP the/DT pound/NN)
  is/VBZ
  widely/RB
  expected/VBN
  to/TO
  take/VB
  (NP another/DT sharp/JJ dive/NN)
  if/IN
  (NP trade/NN figures/NNS)
  for/IN
  (NP September/NNP)
  ,/,
  due/JJ
  for/IN
  (NP release/NN)
  (NP tomorrow/NN)
  ,/,
  fail/VB
  to/TO
  show/VB
  (NP a/DT substantial/JJ improvement/NN)
  from/IN
  (NP July/NNP and/CC August/NNP)
  (NP 's/POS near-record/JJ deficits/NNS)
  ./.)


Man beachte, dass hier das [POS-Tagset der Penn Treebank](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) verwendet wird.  

Wichtig ist auch, dass das "rohe" CoNLL-2000-Corpus im sogenannten **CoNLL-Format** kodiert ist, bei dem jedes Wort und seine Tags eine Zeile bilden:

In [13]:
print(conll2000.raw()[:200])

Confidence NN B-NP
in IN B-PP
the DT B-NP
pound NN I-NP
is VBZ B-VP
widely RB I-VP
expected VBN I-VP
to TO I-VP
take VB I-VP
another DT B-NP
sharp JJ I-NP
dive NN I-NP
if IN B-SBAR
trade NN B-NP
figur


Der Umfang der Chunks wird hier nicht mit Klammern, sondern mit Hilfe der **IOB-Annotation** ("Inside", "Outside", "Beginning") angezeigt. `B-NP` bedeutet etwa, das ein NP-Chunk an diesem Wort beginnt. Das Ende des Chunks wird durch ein mit `O` oder `B` annotiertes Wort markiert. <span style="color:red">(Frage am Rande: Ist die IOB-Notation genauso ausdrucksstark wie die Klammernotation?)</span> 

Im Beispiel von oben sieht so aus, wobei hier nur NP-Chunking durchgeführt wird:

<img src="https://www.nltk.org/images/chunk-tagrep.png" alt="Drawing" style="width: 500px"/>

Praktischerweise sind die ConLL-Daten bereits in Trainings- und Testdaten aufgeteilt:

In [6]:
conlltrain = conll2000.chunked_sents('train.txt')
conlltest = conll2000.chunked_sents('train.txt')

print("Vehältnis Testdaten/Trainingsdaten: ",len(conll2000.chunked_sents('test.txt'))/len(conll2000.chunked_sents('train.txt')))

Vehältnis Testdaten/Trainingsdaten:  0.2251566696508505


### MFC-Baseline

Vergleichbar mit dem POS-Tagging können wir uns erst einmal klar machen, wie schwer die Aufgabe ist, und eine einfache MFC-Baseline ("Most Frequent Chunk") bestimmen. Dafür schauen wir am besten auf die IOB-Annotation im rohen CoNLL2000-Corpus, da hier die einzelnen Wordtoken annotiert sind:

In [7]:
from nltk.probability import FreqDist

chunkdist = FreqDist([chunktag for word,postag,chunktag in conll2000.iob_words('train.txt')])
mfc = chunkdist.most_common(1)[0][0]

chunkdist.most_common(20)
# mfc

[('I-NP', 63307),
 ('B-NP', 55081),
 ('O', 27902),
 ('B-VP', 21467),
 ('B-PP', 21281),
 ('I-VP', 12003),
 ('B-ADVP', 4227),
 ('B-SBAR', 2207),
 ('B-ADJP', 2060),
 ('I-ADJP', 643),
 ('B-PRT', 556),
 ('I-ADVP', 443),
 ('I-PP', 291),
 ('I-CONJP', 73),
 ('I-SBAR', 70),
 ('B-CONJP', 56),
 ('B-INTJ', 31),
 ('B-LST', 10),
 ('I-INTJ', 9),
 ('I-UCP', 6)]

Das häufigste Chunk-Tag ist also `I-NP`. Im Trainingskorpus wird dieses Tag immerhin bei knapp 30% der Wordtoken verwendet:

In [15]:
100 * chunkdist.most_common(1)[0][1] / len(conll2000.iob_words('train.txt'))

29.900296136062003

Dies ist also auch unsere Accuracy-Baseline, d.h. ein sehr einfacher Chunker, der jedes Worttoken mit dem Tag `I-NP` versieht, erreicht eine Accuracy in dieser Höhe. Korrekterweise müssen wir diesen Wert anhand der Testdaten errechnen:

In [16]:
100 * FreqDist([chunktag for word,postag,chunktag in conll2000.iob_words('test.txt')]).most_common(1)[0][1] / len(conll2000.iob_words('test.txt'))

30.34383772716719

Natürlich ist es noch nicht einmal theoretisch brauchbar, jedes Tolken mit `I-NP` zu taggen, weil das zu einem nicht wohlgeformten IOB-Muster führt. Trotzdem wollen wir der Vollständigkeit halber als erstes so einen Default-Chunker implementierten.  

### Default-Chunker

Ein Default-Chunker weist jedem Word dasselbe Chunk-Tag zu. NLTK bietet hierfür keine fertige Implementierung an; allerdings kann man dafür den Default-POS-Tagger aus der letzten Sitzung verwenden. Man muss dafür einfach POS-Tags wie Wortformen und Chunk-Tags wie POS-Tags behandeln:  

In [17]:
# Adapted from https://www.nltk.org/book/ch07.html
class DefaultChunker(nltk.ChunkParserI):
    def __init__(self, default): 
        self.tagger = nltk.DefaultTagger(default)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

Die Chunker-Klassen, die vom Interface `ChunkerParserI` abgeleitet werden, haben eine Methode `evaluate(goldtestdata)`, mit der die Evaluation anhand von Goldtestdaten durchgeführt werden kann:

In [21]:
default_chunker = DefaultChunker('I-NP')
print(default_chunker.evaluate(conlltest))

ChunkParse score:
    IOB Accuracy:  32.6%%
    Precision:      0.4%%
    Recall:         0.0%%
    F-Measure:      0.1%%


Die "IOB Accuracy" liegt zwar in einem Bereich, den wir erwartet haben, stimmt aber nicht exakt mit der oben berechneten Accuracy überein. Auch die Werte für Precision, Recall und F-Measure wirken erst einmal seltsam, weil sie sich erheblich von der IOB Accuracy unterscheiden. 

### <span style="color:red">Aufgaben I: Evaluation</span>

<span style="color:red">A1:</span> Geben sie je einen kurzen Erklärungsversuch für den Wert der IOB Accuracy und für die Werte Precision, Recall und F-Measure. Wie kommen diese Werte zustande und wie kann man deren großen Unterschied erklären? 

#### <span style="color:red">Lösung A1</span>



##### IOB Accuracy:
Die accuracy gibt lediglich den prozentsatz der richtig vorausgesagten Chunk-Tags aus, also

$\frac{richtig klassifiziert}{richtig klassifiziert + falsch klassifiziert}$

Wobei "richtig klassifiziert" aus $True Positive(TP) + True Negative(TN)$ besteht und falsch klassifiziert sich aus $False Positive(FP) + False Negative(FN)$ zusammensetzt.

Also:

$\frac{TP + TN}{TP + TN + FP + FN}$

Der Zähler ist dann einfach der MFC -> TP = 63307 und da wir nur "I-NP" klassifizieren gibt es keine TN =0 und auch keine FN = 0. Der Nenner ist die gesamte Anzahl aller Voraussagen(wort-länge des gesamten corpus) = 211727. FP sind alle Wörter die "I-NP" klassifiziert wurden, aber nicht zu dieser gehören, FP = 211727 - 63307. 

$\frac{63307 + 0}{63307 + 0 + (211727-63307) + 0}$


Dadurch gelangt man an einen hohen %-wert, wenn man das $mfc$ als chunk-Tag nutzt. 
##### Precision:
Die Precision hingegen berücksichtigt im Zähler nur die $TP$ und im Nenner werden lediglich die Anzahl der Positiv klassifizierten samples einbezogen.

$\frac{True Positive}{True Positive + False Positive}$

Daraus ergibt sich:

$\frac{63307}{63307 + False Positive}$

##### Recall:

$\frac{True Positive}{True Positive + False Negative}$

$\frac{63307}{63307 + 0} = 0$


##### F1-Measure:

Und der F1-Score ist ein Wert der aus den beiden obrigen Metriken verrechnet wird:

$F1 = 2\times \frac{Precision \times Recall}{Precision + Recall}$


In [22]:
100 * (chunkdist.most_common(1)[0][1] / (chunkdist.most_common(1)[0][1] + len(conll2000.iob_words('train.txt'))))

23.017881425569204

In [23]:
chunkdist.most_common(1)[0][1]

63307

In [24]:
len(conll2000.iob_words('train.txt'))

211727

### Regexp-Chunker

Wie beim POS-Tagging gibt es für das Chunkging ein regelbasiertes Verfahren mittels regulärer Ausdrücke. Dieses Verfahren benötigt keine Lerndaten, denn es können die Chunking-Regeln auf Grundlage der POS-Tags direkt angegeben werden.

Man unterscheidet vier Arten von Chunking-Regeln:
1. **Chunk:** Fasse die POS-Tags zu einem Chunk zusammen.
2. **Chink:** Fasse die POS-Tags **nicht** zu einem Chunk zusammen.
3. **Split:** Teile das Chunk an einer bestimmten Stelle. 
4. **Merge:** Fasse zwei Chunks zusammen.

Die Reihenfolge der Rege-Typen kann beliebig festegelegt werden. Man kann zum Beispiel erst einen großen Chunk erstellen, diesen dann splitten, dann bestimmte Chunks chinken, dann wieder bestimmte Chunks mergen usw. 

Eine Regel beschreibt eine **Kette von POS- oder Chunk-Tags** mit Hilfe von regulären Operatoren:


| **REGEXP** | **Beschreibung**                                     |
| :-------   | :-------------------                                 |
| `<NN>`     | ein Worttoken mit dem POS-Tag `NN`                   |
| `<NN>+`    | ein oder mehr als ein Wordtoken mit dem POS-Tag `NN` |
| `<NN.?>`   | ein Worttoken mit dem POS-Tag `NN` oder `NNS` oder `NNP`        |
| `<NN\|AT>`   | ein Worttoken mit dem POS-Tag `NN` oder `AT`         |
| `<.*>`     | ein Wordtoken mit einem beliebigen POS-Tag           |
| `<VP><NP>` | eine Verbalphrase gefolgt von einer Nominalphrase |

Der Regel-Typ wird mit geschweiften Klammern angedeutet:

| **REGEXP**           | **Beschreibung** |
| :------------------- | :-------         |
| `{REGEXP}`           | Chunk            |
| `}REGEXP{`           | Chink            |
| `REGEXP}{REGEXP}`    | Split            |
| `REGEXP{}REGEXP`     | Merge            |


Hier ein kleines Beispiel:

In [25]:
grammar = r"""
  NP:
    {<.*>+}          # Chunk everything
    }<VBD|IN|RP>+{   # Chink sequences of VBD, IN, RP
  VP: 
    {<VBD><.*>+}     # Chunk VBD and everything following it
    }<NP>{           # Chink NP chunks 
  """
regexp_chunker = nltk.RegexpParser(grammar)

sentence = [("Rapunzel", "NNP"), ("let", "VBD"), ("down", "RP"), 
                 ("her", "PP$"), ("long", "JJ"), ("golden", "JJ"), ("hair", "NN")]
print(regexp_chunker.parse(sentence))

(S
  (NP Rapunzel/NNP)
  (VP let/VBD down/RP)
  (NP her/PP$ long/JJ golden/JJ hair/NN))


In [26]:
print(regexp_chunker.evaluate(conlltest))

ChunkParse score:
    IOB Accuracy:  47.2%%
    Precision:     26.5%%
    Recall:        13.7%%
    F-Measure:     18.1%%


### <span style="color:red">Aufgaben II: Regexp-Chunker </span>

<span style="color:red">A2:</span> Überlegen Sie sich sinnvolle Chunking-Regeln und implementieren Sie diese wie oben als reguläre Ausdrücke! Führen Sie eine Evaluation mit `evaluate()` durch und versuchen Sie einen möglichst hohen F1-Wert zu erzielen!

In [34]:
# Lösung A2

mygrammar = r"""
  NP: {<DT|JJ|NN.*>+}          # Chunk sequences of DT, JJ, NN
  PP: {<IN><NP>}               # Chunk prepositions followed by NP
  VP: {<VB.*><NP|PP|CLAUSE>+$} # Chunk verbs and their arguments
  CLAUSE: {<NP><VP>}
  """
my_regexp_chunker = nltk.RegexpParser(mygrammar)
print(my_regexp_chunker.parse(sentence))

(S
  (NP Rapunzel/NNP)
  let/VBD
  down/RP
  her/PP$
  (NP long/JJ golden/JJ hair/NN))


In [35]:
print(my_regexp_chunker.evaluate(conlltest))

ChunkParse score:
    IOB Accuracy:  51.2%%
    Precision:     37.6%%
    Recall:        19.6%%
    F-Measure:     25.8%%


In [36]:
# Eine weitere Lösung wäre:
mygrammar = r"""
  NP: {<[CDJNP].*>+}
  """
my_regexp_chunker = nltk.RegexpParser(mygrammar)

In [37]:
print(my_regexp_chunker.evaluate(conlltest))

ChunkParse score:
    IOB Accuracy:  61.9%%
    Precision:     69.7%%
    Recall:        38.0%%
    F-Measure:     49.2%%


### N-Gram-Chunker

Genauso wie beim POS-Tagging kann man auch beim Chunking die Tags der $n-1$ vorangegangenen Worttoken nutzen und dementsprechend Unigram-, Bigram- und Trigram-Chunker entwickeln. Tatsächlich kam dafür wie beim Default-Chunker oben die POS-Tagger wiederverwenden, indem man dafür einfach POS-Tags wie Wortformen und Chunk-Tags wie POS-Tags behandelt. 

In [38]:
# Taken from https://www.nltk.org/book/ch07.html
class UnigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents): 
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.UnigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)
    
class BigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents): 
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

class TrigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents): 
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.TrigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

In der Evaluation fallen zwei Dinge auf: 
1. Es wird kein Backoff benötigt, um eine einigermaßen akzeptable Performanz zu erzielen. <span style="color:red">(Wieso?)</span>
2. Trotzdem fällt die Performanz beim Trigram-Chunker im Vergleich zum Bigram-Chunker wieder etwas ab. 

In [39]:
unigram_chunker = UnigramChunker(conlltrain)
print(unigram_chunker.evaluate(conlltest))

ChunkParse score:
    IOB Accuracy:  86.0%%
    Precision:     73.7%%
    Recall:        85.9%%
    F-Measure:     79.3%%


In [40]:
bigram_chunker = BigramChunker(conlltrain)
print(bigram_chunker.evaluate(conlltest))

ChunkParse score:
    IOB Accuracy:  89.2%%
    Precision:     80.8%%
    Recall:        86.0%%
    F-Measure:     83.3%%


In [41]:
trigram_chunker = TrigramChunker(conlltrain)
print(trigram_chunker.evaluate(conlltest))

ChunkParse score:
    IOB Accuracy:  88.1%%
    Precision:     80.6%%
    Recall:        84.7%%
    F-Measure:     82.6%%


### <span style="color:red">Aufgaben III: Backoff & Datenknappheit</span>

<span style="color:red">A3:</span> Implementieren Sie eine Backoff-Sequenz für die oben benutzen Chunker! (Denken Sie dabei an die Option `backoff`.) 

In [42]:
# Lösung A3
class BackoffChunker(nltk.ChunkParserI):
    def __init__(self, train_sents): 
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data, backoff=nltk.UnigramTagger(train_data))

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)


In [43]:
backoff_chunker = BackoffChunker(conlltrain)
print(backoff_chunker.evaluate(conlltest))

ChunkParse score:
    IOB Accuracy:  89.3%%
    Precision:     80.8%%
    Recall:        86.1%%
    F-Measure:     83.4%%


<span style="color:red">A4:</span> Zeigen Sie, dass die Performanzeinbuße des Trigram-Chunkers gegenüber dem Bigram-Chunker wahrscheinlich durch Datenknappheit zustande kommt! (Tip: Welcher Chunk-Tag kommt denn am häufigsten vor?) 

In [55]:
# Lösung A4 - leider keine Zeit mehr für diese Aufgabe... :(
# Habe ich viele trigamms die zu selten vorkommen --> overfitten

chunkdisttag = FreqDist([chunktag for word,postag,chunktag in conll2000.iob_words('test.txt')])
mfc = chunkdisttag.most_common(1)[0][0]
print(mfc)
chunkdisttag.most_common(20)

I-NP


[('I-NP', 14376),
 ('B-NP', 12422),
 ('O', 6180),
 ('B-PP', 4811),
 ('B-VP', 4658),
 ('I-VP', 2646),
 ('B-ADVP', 866),
 ('B-SBAR', 535),
 ('B-ADJP', 438),
 ('I-ADJP', 167),
 ('B-PRT', 106),
 ('I-ADVP', 89),
 ('I-PP', 48),
 ('I-CONJP', 13),
 ('B-CONJP', 9),
 ('B-LST', 5),
 ('I-SBAR', 4),
 ('B-INTJ', 2),
 ('I-LST', 2)]

### Ausblick: Klassifiziererbasierte Chunker

Die bisher behandelten Verfahren nutzen ausschließlich POS-Tags für das Chunking. Dies führt zu gewissen Einschränkungen, da die nötigen Informationen nicht immer im POS-Tag verfügbar sind. 

Im NLTK-Buch wird folgendes Beispiel gegeben:

    (3)		
        a.		Joey/NN sold/VBD the/DT farmer/NN rice/NN ./.
        b.		Nick/NN broke/VBD my/DT computer/NN monitor/NN ./.

Der Chunker würde hier auf Grundlage der POS-Tags dieselbe Chunk-Struktur ausgeben. Dabei müssen die Sätze unterschiedlich analysiert werden, nämlich so:

    (3')		
        a.		Joey/NN sold/VBD (NP the/DT farmer/NN) (NP rice/NN) ./.
        b.		Nick/NN broke/VBD (NP my/DT computer/NN monitor/NN) ./.

Dem Chunker fehlt hier die Information, dass *sold* ditransitiv und *broke* transitiv ist. Außerdem ist *computer monitor* eine sogenannte **Kollokation**, d.h. eine geläufige komplexe NP, und *monitor* kann allein beine NP bilden. 

Um Wortformen zu berücksichtigen, aber dabei nicht in ein Sparse-Data-Problem zu geraten, bieten sich sogenannte klassifiziererbasierte Ansätze an. Im NLTK-Buch ist als Beispiel ein Naive-Bayes-Klassifizierer angegeben, bei dem die Features flexibel erstellt werden können. Hier der nur leicht veränderte Code und die Evaluation:

In [56]:
class ConsecutiveNPChunkTagger(nltk.TaggerI):
    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = npchunk_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        #self.classifier = nltk.MaxentClassifier.train( 
        #    train_set, algorithm='megam', trace=0)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = npchunk_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

class ConsecutiveNPChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in
                         nltk.chunk.tree2conlltags(sent)]
                        for sent in train_sents]
        self.tagger = ConsecutiveNPChunkTagger(tagged_sents)

    def parse(self, sentence):
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)

In [57]:
def npchunk_features(sentence, i, history):
     word, pos = sentence[i]
     if i == 0:
        prevword, prevpos = "<START>", "<START>"
     else:
        prevword, prevpos = sentence[i-1]
     if i == len(sentence)-1:
         nextword, nextpos = "<END>", "<END>"
     else:
         nextword, nextpos = sentence[i+1]
     return {"pos": pos,
             "word": word,
             "prevpos": prevpos,
             "nextpos": nextpos, 
             "prevpos+pos": "%s+%s" % (prevpos, pos),   
             "pos+nextpos": "%s+%s" % (pos, nextpos),
             "tags-since-dt": tags_since_dt(sentence, i)}  

def tags_since_dt(sentence, i):
     tags = set()
     for word, pos in sentence[:i]:
         if pos == 'DT':
             tags = set()
         else:
             tags.add(pos)
     return '+'.join(sorted(tags))

In [58]:
chunker = ConsecutiveNPChunker(conlltrain)
print(chunker.evaluate(conlltest))

ChunkParse score:
    IOB Accuracy:  93.0%%
    Precision:     87.2%%
    Recall:        91.9%%
    F-Measure:     89.5%%


Wir sehen, dass der Ansatz mit Bayes-Klassifizierer nochmals deutlich bessere Ergebnisse ermöglicht als die einfachen N-Gram-Chunker.

Trotzdem liegt das noch gut 4 %-Punkte unter dem F-Measure der besten Chunker im CoNLL-Shared-Task vor bald 20 Jahre ...